In [1]:
# import required libraries
import pandas as pd
import datetime as dt

In [2]:
# import January data from CSV into DataFrame
df = pd.read_csv('C:/Users/Ken/Desktop/listings_jan.csv')

In [3]:
# check top three rows of dataframe
df.head(3)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,24018771,https://www.airbnb.com/rooms/24018771,20240108030557,2024-01-08,city scrape,Home in Toronto · ★4.74 · 1 bedroom · 1 bed · ...,NaN,NaN,https://a0.muscache.com/pictures/1030ee76-f3f7...,179038571,...,4.78,4.77,4.72,STR-2009-FDSXPB,f,3,0,3,0,1.83
1,50680103,https://www.airbnb.com/rooms/50680103,20240108030557,2024-01-08,city scrape,Boat in Toronto · 1 bedroom · 1 bath,NaN,NaN,https://a0.muscache.com/pictures/91f84c43-e010...,396389036,...,NaN,NaN,NaN,Exempt,t,1,1,0,0,NaN
2,1018845223985730763,https://www.airbnb.com/rooms/1018845223985730763,20240108030557,2024-01-08,city scrape,Townhouse in Toronto · ★New · 1 bedroom · 1 be...,NaN,NaN,https://a0.muscache.com/pictures/f1e047ed-949d...,9660032,...,NaN,NaN,NaN,NaN,t,4,1,3,0,NaN


In [4]:
# checking number of rows and columns
df.shape

(20386, 75)

In [5]:
# decided to select these columns
df_jan = df[['last_scraped', 'host_id', 'host_name', 'host_since', 'neighbourhood_cleansed', 'room_type', 'price', \
             'minimum_nights', 'availability_30', 'availability_60', 'availability_90','availability_365']]

In [6]:
# checking column data types and also checking if there are null values
# price needs to be converted to 'float' and last_scraped converted to 'datetime' 
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20386 entries, 0 to 20385
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   last_scraped            20386 non-null  object
 1   host_id                 20386 non-null  int64 
 2   host_name               20384 non-null  object
 3   host_since              20384 non-null  object
 4   neighbourhood_cleansed  20386 non-null  object
 5   room_type               20386 non-null  object
 6   price                   16595 non-null  object
 7   minimum_nights          20386 non-null  int64 
 8   availability_30         20386 non-null  int64 
 9   availability_60         20386 non-null  int64 
 10  availability_90         20386 non-null  int64 
 11  availability_365        20386 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 1.9+ MB


In [7]:
# to remove SettingWithCopyWarning
df_jan = df_jan.copy()

In [8]:
# remove '$' and ',' string characters from the 'price' column and then convert it to 'float'
df_jan['price'] = df_jan.loc[:, 'price'].str.replace('$', '').str.replace(',', '').astype(float)

In [9]:
# ensuring data type successfully changed to 'float'
df_jan['price'].dtype

dtype('float64')

In [10]:
# create 'month' and 'year' columns, extracted from 'last_scraped' column
df_jan['last_scraped'] = pd.to_datetime(df_jan['last_scraped'])
df_jan['date'] = df_jan['last_scraped'].dt.strftime('%B') + ' ' + df_jan['last_scraped'].dt.strftime('%Y')
df_jan.head(2)

,last_scraped,host_id,host_name,host_since,neighbourhood_cleansed,room_type,price,minimum_nights,availability_30,availability_60,availability_90,availability_365,date
0,2024-01-08,179038571,Yongmei,2018-03-17,Hillcrest Village,Private room,45.0,1,14,44,74,163,January 2024
1,2024-01-08,396389036,Turan,2021-04-10,Waterfront Communities-The Island,Entire home/apt,180.0,28,30,60,90,365,January 2024


In [11]:
# dealing with NaN values in 'price' column:
# first checked the actual CSV file to manually go to the link of the AirBnB listing of the top 10 after filtering for 
# empty cells based on 'price' column. Noticed in the first 10 listings that you couldn't book them, so looked at
# availability 30-365 days in future. Decided to filter based on empty price and also 0 days availabilty from 30-365
# as these listings could not be booked even up to one year in the future.

# create boolean mask to get rows that have NaN values for price column and 0 days for availability 30-365 days in future.

mask = df_jan['price'].isna() & (df_jan['availability_30'] == 0) & (df_jan['availability_60'] == 0) & (df_jan['availability_90'] == 0) & \
    (df_jan['availability_365'] == 0)

In [12]:
# apply negation of boolean mask 
df_jan = df_jan[~mask]

In [13]:
# check distribution of prices and min/max values
df_jan['price'].describe()

count    16595.000000
mean       177.039168
std        333.108977
min          8.000000
25%         75.000000
50%        120.000000
75%        195.000000
max      12400.000000
Name: price, dtype: float64

In [14]:
# fill NaN values with median as the max value is too high to use mean which could skew results
df_jan.fillna(df_jan['price'].median(), inplace=True)

In [15]:
# make sure no nulls
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16707 entries, 0 to 20385
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   last_scraped            16707 non-null  datetime64[ns]
 1   host_id                 16707 non-null  int64         
 2   host_name               16707 non-null  object        
 3   host_since              16707 non-null  object        
 4   neighbourhood_cleansed  16707 non-null  object        
 5   room_type               16707 non-null  object        
 6   price                   16707 non-null  float64       
 7   minimum_nights          16707 non-null  int64         
 8   availability_30         16707 non-null  int64         
 9   availability_60         16707 non-null  int64         
 10  availability_90         16707 non-null  int64         
 11  availability_365        16707 non-null  int64         
 12  date                    16707 non-null  object     

In [16]:
# drop unnecessary columns (availabilities) and last_scraped as we got what we needed from them
df_jan.drop(columns=['last_scraped', 'availability_30', 'availability_60', 'availability_90', 'availability_365'], inplace=True)

In [17]:
# create a csv import function to clean and modify the other five csv files in the same manner so that I can join them into one
def csv_import(csv_location):
    """
    Import csv file with same criteria
    """
    df = pd.read_csv(csv_location)
    df = df[['last_scraped', 'host_id', 'host_name', 'host_since', 'neighbourhood_cleansed', 'room_type', 'price', \
             'minimum_nights', 'availability_30', 'availability_60', 'availability_90','availability_365']]
    df = df.copy()
    # replace '$' and ',' in 'price' column and change datatype to 'float'
    df['price'] = df.loc[:, 'price'].str.replace('$', '').str.replace(',', '').astype(float)
    # change 'last_scraped' column to datetime in order to extract month and year
    df['last_scraped'] = pd.to_datetime(df['last_scraped'])
    df['date'] = df['last_scraped'].dt.strftime('%B') + ' ' + df['last_scraped'].dt.strftime('%Y')
    # filtering rows based on NaN values in 'price' column along with 0 days available from 30-365 days in future
    mask = df['price'].isna() & (df['availability_30'] == 0) & (df['availability_60'] == 0) & (df['availability_90'] == 0) & \
    (df['availability_365'] == 0)
    df = df[~mask]
    # fill NaN values with median value of 'price' column
    df.fillna(df['price'].median(), inplace=True)
    # drop unnecessary columns
    df.drop(columns=['last_scraped', 'availability_30', 'availability_60', 'availability_90', 'availability_365'], inplace=True)
    
    return df

In [18]:
# use function for December data
df_dec = csv_import('C:/Users/Ken/Desktop/listings_dec.csv')

In [19]:
df_dec.head()

,host_id,host_name,host_since,neighbourhood_cleansed,room_type,price,minimum_nights,date
0,273730029,Tran Thien Ly,2019-07-05,Humber Heights-Westmount,Private room,72.0,28,December 2023
1,474878080,Huu-Hoa,2022-08-13,Downsview-Roding-CFB,Private room,110.0,28,December 2023
2,141166875,Andrew,2017-07-18,Kingsview Village-The Westway,Private room,104.0,1,December 2023
3,8569918,Sharon,2013-09-01,Leaside-Bennington,Private room,120.0,28,December 2023
4,537627593,Qing,2023-09-17,Banbury-Don Mills,Private room,120.0,2,December 2023


In [20]:
# making sure it ran correctly and no NaN in the price column
df_dec.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19070 entries, 0 to 19993
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   host_id                 19070 non-null  int64  
 1   host_name               19070 non-null  object 
 2   host_since              19070 non-null  object 
 3   neighbourhood_cleansed  19070 non-null  object 
 4   room_type               19070 non-null  object 
 5   price                   19070 non-null  float64
 6   minimum_nights          19070 non-null  int64  
 7   date                    19070 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 1.3+ MB


In [21]:
# run it for the rest of the months
df_nov = csv_import('C:/Users/Ken/Desktop/listings_nov.csv')
df_oct = csv_import('C:/Users/Ken/Desktop/listings_oct.csv')
df_sep = csv_import('C:/Users/Ken/Desktop/listings_sep.csv')
df_aug = csv_import('C:/Users/Ken/Desktop/listings_aug.csv')

In [22]:
# join all dataframes into one
df = pd.concat([df_jan, df_dec, df_nov, df_oct, df_sep, df_aug])

In [23]:
# check for room types and remove 'Hotel room' as there is only one each month, and it is listed by the same host
print(df['room_type'].unique())
df[df['room_type'] == 'Hotel room']

['Private room' 'Entire home/apt' 'Shared room' 'Hotel room']


,host_id,host_name,host_since,neighbourhood_cleansed,room_type,price,minimum_nights,date
19471,124915178,Jaime,2017-04-08,East End-Danforth,Hotel room,38.0,28,December 2023
15651,124915178,Jaime,2017-04-08,East End-Danforth,Hotel room,38.0,28,November 2023
7002,124915178,Jaime,2017-04-08,East End-Danforth,Hotel room,38.0,28,October 2023
6991,124915178,Jaime,2017-04-08,East End-Danforth,Hotel room,38.0,28,September 2023
7012,124915178,Jaime,2017-04-08,East End-Danforth,Hotel room,38.0,28,August 2023


In [24]:
# drop rows with room_type equal to 'Hotel room'
mask = df['room_type'] == 'Hotel room'
df = df[~mask]

In [25]:
# reset DataFrame index
df = df.reset_index(drop=True)

In [26]:
# checking total number of rows and columns
df.shape

(120517, 8)

In [27]:
# make sure no NaN values
# dataset ready for Tableau
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120517 entries, 0 to 120516
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   host_id                 120517 non-null  int64  
 1   host_name               120517 non-null  object 
 2   host_since              120517 non-null  object 
 3   neighbourhood_cleansed  120517 non-null  object 
 4   room_type               120517 non-null  object 
 5   price                   120517 non-null  float64
 6   minimum_nights          120517 non-null  int64  
 7   date                    120517 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 7.4+ MB


In [28]:
# export to CSV 
df.to_csv('C:/Users/Ken/Desktop/airbnb.csv', index=False)